# CBA Policy Portal Sraper
CBA make downloading the policy difficult by
1. Having a SSO based login that makes request-http automation difficult
2. Using ASPX to dynamically load content.

Oover come this using by loggign in with the browser, then ctting and pasting the auth cookies into this notebook.

Scraper Code
1. Only searchs for links in div#wrapper
2. Has code to deal with ASPX/DynamicContent
3. Limited to commbroker domain

In [1]:
from  scraper import WebScraper
from loguru import logger
import os

class CBA(WebScraper):
    def __init__(self, domain:str, cookie:str, download_path:str, search_only:bool=False):
        super().__init__(domain, cookie, download_path, class_names = ["div#wrapper"])
        self.search_only = search_only

    def scrape(self, url:str, depth:int, starting_page:bool = True) -> None:

        logger.info(f"Scraping {depth}: {url}")
        
        if self.skip(url, depth, ["Logon.aspx"]):
            return
             
        _, extension = self.get_path(url)
        assert extension in [".html", ".aspx"], f"Invalid extension '{extension}' for {url}"

        # If aspx find the HTML url it refers to
        if extension == ".aspx":
            html_url = self.aspx_link_to_html_link(url)
            if not html_url:
                return
        else:
            html_url = url

        # read the contents of the html_url to the original url
        # so ASPX paths are used for the HTML
        html_content = self.fetch_content(html_url) 
        html_content = self.remove_script(html_content) 
        if not (starting_page or self.search_only):
            self.save_content(url, html_content)
        
        # extract all the links and recurse
        self.extract(html_content, depth, "div#wrapper")

# Download Comm Broker

In [2]:
import shutil

directory_path=r"/home/kevinmcisaac/Projects/policy-pal-pages/policy/CBA/CommBroker/"

if os.path.exists(directory_path):
    shutil.rmtree(directory_path)  
os.makedirs(directory_path) 

#Need depth 3 to get to 
cba = CBA("https://www.commbroker.com.au", 
              cookie="s_cc=true; at_check=true; SMT_TRACK_PP=nb:customerportfolioview; kndctr_FFF9306152D80A5C0A490D45_AdobeOrg_identity=CiY1MzQ4ODc3MDE3MzAyMTkyMTI1MzA4NTU5MDY0NjE3NTIxMzI2NlITCJei08XDMhABGAEqBEFVUzMwAPABl6LTxcMy; AMCV_FFF9306152D80A5C0A490D45%40AdobeOrg=MCMID|53488770173021921253085590646175213266; s_gnr=1736118160971-New; ADRUM=s=1736119164952&r=https%3A%2F%2Fwww.commbroker.com.au%2Fapp%2Fcpv%2Fportfolio-loans%3F0; s_nr=1738999017033; s_sq=%5B%5BB%5D%5D; bm_mi=485C5D0C50585DEF124E4B3C63DDE00B~YAAQXgNiaPFyCdmUAQAAmUb+5xoXhD90RnsA86R/8YaLtEGnr/PvUhMKWxrnVYWig9VBJKL9YrEMlSiQz4Hsea+hskQFIVZD3D0OzdMWM+MPqwIvA5gcXstg5U0aq50B+EZFnCvGpahTvF/6IXYBgIi4wZuZI/XSS/RgdHCxnKo4pS3cf1YQke1OsN9YSyXdqnLoK8yE7JJfQZ7WcJIkQnw4ETyJKVowfOKFJuS91nlmE2XeRZTND4j9vB1t1Yf5p//YhuL3Er8kQW72PFdvAwZvIAr5kn9ifct5CFZZmYh83GPO/mbbu1eo9hXgczPf5fx6HQcV1Yhdecr8eWpuo4pAWEbkXP9ckjFOgW8QSxQ=~1; ak_bmsc=1669B8F9C2257421ED9337A2846FAC5D~000000000000000000000000000000~YAAQRQNiaOYQMtiUAQAAb07+5xqENOjeCaDMiGx8u9tPTdDHNqXsZxr3zM2vSfxjxoGa5SOfamjcR2b0W7jc6yaTSoKo8VaUb8toAxAKwzN5Yhg8IUjqWHQJ15JazflFFgtn6UFYiDSU1B9zFTfYAh1rFKFmajK8kU6ic0Oo/u86WOKqeN0XwzyJKOnitDUyRRjVspkQ3Ao2dos04dmoFeltu/5mJiu9iANibPGyCdXK3Ss8di9uUX8ELL5y3oj6WxOzGSVx2+pmWrGwsHHau9P+eWC3li2vPkt/P7C/gGloF87a0K1L7JHJhMcacVKNe5CgYHYK9c96Tb/AGctG7lH/pjIwk9K3C1+xVkMd0iEDDft2w3fBNzKARnkmBZjEqdTix0rM9NlvEWqIKgQjXfGuMC1mSanRtg/N0T14lRRnfJbhmyHYWXdX7mDJidxho2LAyRIE6G/0FBcAcAdLGpZavcOVQCJ1wystl2cZBe3BI7E1lIZudozFoCVjbsaSJ9/L6OWflKI=; ASP.NET_SessionId=0u5ztwpwcnashvfzb5h0srx0; cbid=8i6Xd03Vaqi_WH0E1HTJiTYjiIiClEPYCNDY6D_GVaVKNWFVWwucPh_mvvEZJlqVErqxL0Ohd4ABelx0m63bg8t9g3W7_82gfud4GaHyRmtA8f53G0OY_v8hck1axe-UEPt_UIxxj7z-r8mX1n94YZ7F0gX5UC4QgfmTMfsj-DU0_J1Pa7IffvvGafTHApZm7-l1p2YtnCaeNS9sbXUzaTp6TFfcKxQRd-AfXyeYB9w-SV66bsD8KcNVn2_F0wnrJqd-7YcEB_m0KXBV7AQbWxWRLd3AJBTEcqFN2iE2zzvaI3lkrR_IAOiCQPzZ12AYsFddjJQ4sov2smDd_zYCadJFeTQMS6zvlmwDuRPHPgCB9uJqcs1AsyHK9ixE7_gJ0jYFIURGXYi-DyQDcTZfBtvF3plI4CkbsCLoUBPldkZa1V1pnXlO8buFdc0gxB8UgHvECjyRFKcJS_kyE_Q1wDtJKBZbavW-_8t1VFEdQpW5OcG6SpKGxj1neAUtg5zC0FtgI_RQ26eUX0xCFYONWSWathClsCwWOdcTf_49oeFJ0IkpY9-d6kDjJ4uZ1zEnv0yEKuLFqasffH6IJ9gCEt30Iwql_7UO3d4I_TN0rjPpymTyY1CD7TvkhOlMZRxQ76KkhvLvAwUT3umblySZ3nWdN9b6VakA-g9RhwuWXweH3j67FSCzJQgQScsA65K9B-LFg8Mfdte62flijIq13H3BX_R1llQZGByodFmopDvUHfr-Q7UC7lW75EZitvtQCznio1LYVqNHWks6UGg-sJCQ_c2Yr5-vE7qiv0s69X9j5RpRo2g7i8-dqJQVgsA4xqwVmraLMBzc5RrLbC5V2_EEI8RYBDfzR6YW9djH-ykSwuT7qvy42CgpR8hwnhqAnzEPVb7xzeyHedZdA-_CREixnh4-_DYIvYhf_MpGJR41YC6RlXB7374dcBBZIntyDnAhxhmJBaaDccfq89LgFTuiqNp8G2p9VIebgL3ZzHVaREy5iksIQv7s63W-spaGi16BSuwSgrsv0BHzaX6YQakhrQZET44p0U5iM5fsetyPWkkUKB021cQUwr7tUs8EYXaOrmHXaJBzFW0qvQ3cghtixn1SGraQEytMFQ265I5t_9dbBkdRG4aQzxju-zuWq9rIibv8IJXvXNPD8b_he7ceehIdylaxk88WaDT7wfA5Mfbp1N_ML-ITJP4q7BmJCo6harVkbb1y1s-BDdbhzZQg2c8X5kDirIiaVj5dpmvZBrfkPNpT3P-3JwdhFd1rY8dfa4vrWKkRkfhnsjDcUwRFZGO7dyD52TRRlbyzP2pyAIFREd2XAKscCRuXRFaM8380pYyWgIITUvPQB42ih-147VNvZ5W_ucpeZAz94FiuCiNIOZMBIpv0ccSG5Js7ASHbD6e39EmhxLFuf-8SnX_Q_7PoQAPxSUgFOH-kQGg-3Mv_trRi_9Ch2hGS6zAvkcV9rToOdyiTqZcnvo_93mkp9RdKBcIGH_gbycPw21_W-UnAdge0VPgsmoFGX37g5S7K02Onr2i6LC5FBhhuBMeuq9jc1bGA5aTL_PsnmDtV3xAVB9fwCb-vDtuzbflRCwvr0ykHXztENBoA83w4RClR7TZEuJp-L7815lxYNCTqZbUuDFNwngMAJf9qbgTRT1F7hpIHXaTBdj0qBmGxK0bmXYH57sihpAHFiX3DyMQ81O-4Nk2-gY8tDOBUyjTHLTtLDU4GZNzRNAIrbI-WBrpjCyoPvEDDXRPTMhpcYUOhOpMAFPjUVT__ehCvq7vJ3U6DLh5YoUsV99b9GkoQeFShNLlVXSABLkAulRBugbivX-RR7AqC-YCc3cgpNqfQ3l8QT_y42lF2aOSlhaF5MofIYYQM-fK7neqBRd9nEzv6TtfVe0waRx6up3va_aYaSS2vI_ta-_93zQxqS-iPMDE8pu_wAxjiDzm1MNAb8FyZsHe2Re1BlJHC2zcAp1WW5guj8253QhW2pgCO6k9dJuur0CAtZCWUKJTvZ3Eme5xK3Y19H3K30yrsijoPEfLuZ1D0PfvVRATghmfk238IQIA9Veu9cDWgL4pYEkTibObtNRAYyBg8sJMZoWSpd1Msat67iKAjGoJBL6t0rGRk-PyxMzKbhuelEF8YLjyjymJBFhnMDt2pmLDhvpuWDq66i3MPGqlQMhb60MNSiBCoK7ibeycYnro5h1s8TdBHNvDsnVCR752EeF1gFf2Ys4PGn87O1F20kD08pib-PNwDrBDJaF7yWUt0mSIYgUreAnuLMzgwE0s4qbpxkmitpJU14HuEbHCPiJvV7JYDroKtbWsoYc5NvaEVSvephWxjQB1Ty99kDdleRe8LppJ6awsjTayZe71E8I5UjVvDAPZw8--O2IYo9C3Hf0GVUWibk_50WPj3UEMoxfY30urIO1MKIW5GzNuBJMQ_8tuMNiYwgQ7ZOOq4UhRmVNeWP_L2UCfpJi3TfLFxd6XMZkP5eliRh9RAIpl3a_c7gmXMCz99hsb6WSrp99qQqp0siwFMdSUYyXznFR3vPvGUEZNVME-Ojuro_AIH84XnuJeYejOlFOQR5tkRIR5Wrr3QSfDTYkwt7zvtfkfH0LfcE4v_sPYzOtLJjicqhWFS0XsazQ2MBzEE1raZ_4D_OAEhsqOsGoUgiQmA89UpykLLzacyYqDs4Li3P7eAcFdu-mNl0OnC_rwAmFRKlEP0uM1si4i2u4T0G0MTyZeyafoI_ITGecjPxsry1uhC8Yo0V4UujCBTkshBlgDFGFfhpJxBsk9AaeXnD4R8VsWA-kIK4WMMHcEkPaCNWTbqI7aa2dmy2u12DOwwq885xz8on8Yekhx7diKRbw5XVg55YxTcnM_lnGqv3c6SAROBISAAFaYUWaRKr8CW9rOIJOqSL4xRRu7JS7olcpAVkXn_f-g-K8NawAkJrz71gfJ-u9ID6-rzQldqlC0LGqfc_pBlwVAgcuzX8GzqWEq4453fpz0DIFvC2Xm2qeBZ35p5Xa_qJDU_NnSs71pCCCdcRPPD_TDzmqTyt81GKnZUy-pOv0GsIADpWbOnQareb0SXxLLMx82qAgk8U5uyHPndjl9HJbG9VeKfdvOH9I7IIcL5rhspjWrV2MEHpQ_CUzsWG6gSTNABc4cyIdLmbywj3dsz29yiB3D4ML1c_MsHAqmR3ddvtfxwAikbEIrEouf_vfbVX0qAdwjiG9Agj8Uy4eWYGTI5m2mILOkHTyBJ2i1V4afSBmRJqMMobHxWA5TgVKsgN18WYpd-vlFcpZMxySS93bbekgxc9Jl0sBD2gg8gpcmi3IziiZL-G6tHwbmU9LfczyhFsQ-fmfKFXT4hFHhHVnYhnxzXmVYjbpsU-5b3zP_g7wj5dboxH0gYaQ5btQpS6tTRidodsdXHXBXASbGC92-DpFvYcrCc-fxLm3e6nbqm2_0PchuIrFj79A; bm_sz=62E209789FF31088A51353BB1D3BF1D0~YAAQ3ETHF5XICayUAQAAL6YL6BoToLEehwUPT/BvvXggRzSEDimQGM+rTcyJAmSyxOj8KiA/VXGv3hXlekKSMGbxrG7sjHaNp5hjzBLzaSjvWQFnynI8VuYCMPAyqnQ6erRHg2ij9y/zcQADFTIZzyTHCi1H+vlxyT7pIQwiHpbcG4gSTSkV4OOEQx4P9zZzLE0azPtGuRbwNBsfMfPxumtIZH2gbwbN0NmNV3CDGbdGNOqpZshSdpsjBVaJJavltAuovGhZDC7PmmmKoIzjIcwxneajT7o/8OUnrHWe6ZYWZswP6RE2ZeqHGkLSTaXev08LW0gI8NKl9BrGN20zi0hgurki+EhMnoQ1to66Er7NT/dzcOX13v//2XWpzERVXdBHn9jxekdOA5V9MQozTNMiPyVe+eR72b8o905W8x6jwns/sEcWx2ur4BD3hpo2BP+neIZWXGcbz01EzVDm~4473924~3490372; htsm=exp=1739061670~acl=/*~hmac=1c71a5aecc64b84b750f1b24433b6840096157d85860bb291e7ca151fb8f026f; bm_sv=A5D36708DC898F0DF0BA6833894B2AD6~YAAQ3ETHF7HLCayUAQAA2fkM6BpbPKws40OEijyO0bOk7S0yh91Wt9pLjK6YchkcF0CrZFoV3K2YarX5Xl6G+XgkKANmzXWLruRyKPxio+okY8tHFm9GIMa2JJyW7ohd5hCRn2vYPArpqFeIQ5QTqwiQDDRgrvX0gu7rEN1Csr8tqP90qAp9gtY89k6W+eXVT8L8L2ECuwFR83p+sVAq7jwn7olMeAmozlxLLjSM6j14yHGZ6WUorIzS9VOU+jNsiD/4JpwphkE=~1; _abck=E63113F89F6BC5685052079ADEBA5708~-1~YAAQ3ETHFwLMCayUAQAAhBEN6A1cX4XHlQnrEhq7ZLNNFBHZIwxZm3cyY6hxJWYWUwrLKq7g6DhkQS+Pr+dUCZwFVggl9JA/Ne5XsaHsDEf2BHNXCP5YVOSmOGFp8N47GXFX+I8JXvkfBd7W2tFDFmiuiKkyzw74uFmcoN4Lx5cbodZh3H9I8btfWSvXh4pX+D5J6Y4zO8X97HOwsQj75VFRbQVQtNs3g+++SC3bSn5g2nL39nJqHdj3ZnVEDGpZHV7KS19UNPpFUZWWBNqGTnZIDJA143rBzGgTVPs+2Wm7C4gUYzPbTwdM7in+0/8cvlQBAiREdrfT/tz0fWSSI7LPRajYs5d6GEfC2RJWHLi3oP0InIFVYp/3XJAqKyF7NVlr8oIfEsGPCfQbnR95iP5CkUGD9KS+vmMiPJ3LTDGGKagck/Rkb1t7W6pRhF/MLcbemW5eohNN7NtQDQN3q/P9B18UY9KtM3ZY6CDAJYNfTd8CJOhmVuu8hKK0lvOK9moa~-1~-1~-1", 
              download_path= directory_path, 
              search_only=False)
cba.scrape("https://www.commbroker.com.au/Net/Documentum/credit-policy/default.aspx", depth=16)
cba.scrape("https://www.commbroker.com.au/Net/Documentum/processes/default.aspx", depth=16)

print(len(cba.download))

2025-02-09 08:25:15.374 | INFO     | __main__:scrape:12 - Scraping 16: https://www.commbroker.com.au/Net/Documentum/credit-policy/default.aspx
2025-02-09 08:25:16.573 | INFO     | __main__:scrape:12 - Scraping 15: https://ecomms.cba.com.au/cv/4e28c09365e1c179bd89ab131200ec829530c314
2025-02-09 08:25:16.574 | INFO     | scraper:skip:130 - Skipping domain https://ecomms.cba.com.au/cv/4e28c09365e1c179bd89ab131200ec829530c314
2025-02-09 08:25:16.576 | INFO     | __main__:scrape:12 - Scraping 15: https://www.commbroker.com.au/Net/Documentum/credit-policy/acceptable-excluded-purposes.aspx
2025-02-09 08:25:16.970 | INFO     | scraper:save_content:56 - Downloading https://www.commbroker.com.au/Net/Documentum/credit-policy/acceptable-excluded-purposes.aspx to /home/kevinmcisaac/Projects/policy-pal-pages/policy/CBA/CommBroker//Net/Documentum/credit-policy/acceptable-excluded-purposes.aspx
2025-02-09 08:25:17.012 | INFO     | __main__:scrape:12 - Scraping 14: https://www.commbroker.com.au/Net/Doc

247
